In [1]:
%%sh
rm -r sample_data
pip install --upgrade pandas
pip install --upgrade geopandas
pip install --upgrade mpld3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive

import os

import pandas as pd
import numpy as np
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore', message = '.*initial implementation of Parquet.*')
warnings.filterwarnings('ignore', message = '.*Geometry is in a geographic CRS.*')

import matplotlib.pyplot as plt
import seaborn as sns

import mpld3
from mpld3 import plugins

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pasta_out = 'drive/MyDrive/Aprendizado_Maquina/Projeto_IBGE/dados_tratados'

# Visualizações

In [5]:
gdf_malha = gpd.read_parquet(f'{pasta_out}/malha_setores_censitarios_2010.parquet')
df_info = pd.read_parquet(f'{pasta_out}/informacoes_setores_censitarios_2010.parquet')

lista_df_info_raio = []
lista_raios = [500, 1000, 2000, 4000]
for raio in lista_raios:
    lista_df_info_raio.append(pd.read_parquet(f'{pasta_out}/informacoes_{raio}m_setores_censitarios_2010.parquet'))

In [6]:
lista_geocodi = gdf_malha.loc[(gdf_malha['NM_ESTADO'] == 'SÃO PAULO') & 
                              (gdf_malha['NM_MUNICIP'] == 'SÃO JOSÉ DO RIO PARDO'), 'CD_GEOCODI'].values

gdf_malha_plot = gdf_malha[gdf_malha['CD_GEOCODI'].isin(lista_geocodi)]
df_info_plot = df_info[df_info['CD_GEOCODI'].isin(lista_geocodi)]
lista_df_info_raio_plot = []
for df_info_raio in lista_df_info_raio:
    df_info_raio_plot = df_info_raio[df_info_raio['CD_GEOCODI'].isin(lista_geocodi)]
    lista_df_info_raio_plot.append(df_info_raio_plot)

gdf_plot = pd.merge(gdf_malha_plot, df_info_plot, how = 'inner', on = 'CD_GEOCODI')
for df_info_raio_plot in lista_df_info_raio_plot:
    gdf_plot = pd.merge(gdf_plot, df_info_raio_plot, how = 'inner', on = 'CD_GEOCODI')

In [113]:
class MouseDoubleClick(plugins.PluginBase):
    
    JAVASCRIPT = '''
    mpld3.register_plugin("mousedoubleclick", MouseDoubleClick);
    MouseDoubleClick.prototype = Object.create(mpld3.Plugin.prototype);
    MouseDoubleClick.prototype.constructor = MouseDoubleClick;
    MouseDoubleClick.prototype.requiredProps = [];
    MouseDoubleClick.prototype.defaultProps = {
        link: "https://www.google.com.br/maps/search/"
    };
    function MouseDoubleClick(fig, props) {
        mpld3.Plugin.call(this, fig, props);
    }

    MouseDoubleClick.prototype.draw = function() {
        var fig = this.fig;
        var link = this.props.link;
        for (var i = 0; i < this.fig.axes.length; i++) {
            var abrir_mapa = function() {
                var ax = fig.axes[i];
                return function() {
                    var pos = d3.mouse(this), x = ax.x.invert(pos[0]), y = ax.y.invert(pos[1]);
                    var url = link + y.toString() + "," + x.toString();
                    window.open(url, '_blank').focus();
                };
            }();
            fig.axes[i].baseaxes.on("dblclick", abrir_mapa);
        }
    };
    '''

    def __init__(self, link = 'https://www.google.com.br/maps/search/'):
        self.dict_ = {'type': 'mousedoubleclick',
                      'link': link}

In [114]:
class ZoomCustomizado(plugins.PluginBase):
    
    JAVASCRIPT = '''
    mpld3_Axes.prototype.bindZoomCustomizado = function() {
        if (!this.zoom) {
            this.zoom = d3.zoom().filter(function() {
                return !d3.event.button && d3.event.type != "dblclick";
            })
            this.zoom.on("zoom", this.zoomed.bind(this));
            this.axes.call(this.zoom);
        }
    };

    mpld3_Axes.prototype.unbindZoomCustomizado = function() {
        if (this.zoom) {
            this.zoom.on("zoom", null);
            this.axes.on(".zoom", null);
            this.zoom = null;
        }
    };

    mpld3_Axes.prototype.isSomeTypeOfZoomEnabledCustomizado = function() {
        return this.isZoomEnabledCustomizado;
    };

    mpld3_Axes.prototype.enableOrDisableZoomingCostumizado = function() {
        if (this.isSomeTypeOfZoomEnabledCustomizado()) {
            this.bindZoomCustomizado();
        } else {
            this.unbindZoomCustomizado();
        }
    };

    mpld3_Axes.prototype.enableZoomCustomizado = function() {
        this.isZoomEnabledCustomizado = true;
        this.enableOrDisableZoomingCostumizado();
        this.axes.style("cursor", null);
    };

    mpld3_Figure.prototype.enableZoomCustomizado = function() {
        this.axes.forEach(function(axes) {
            axes.enableZoomCustomizado();
        });
    };

    mpld3.register_plugin("zoomcustomizado", ZoomCustomizado);
    ZoomCustomizado.prototype = Object.create(mpld3.Plugin.prototype);
    ZoomCustomizado.prototype.constructor = ZoomCustomizado;
    function ZoomCustomizado(fig, props) {
        mpld3.Plugin.call(this, fig, props);
    }

    ZoomCustomizado.prototype.draw = function() {
        this.fig.enableZoomCustomizado();
    };
    '''

    def __init__(self):
        self.dict_ = {'type': 'zoomcustomizado'}

In [117]:
class ResponsivoCustomizado(plugins.PluginBase):
    
    JAVASCRIPT = '''

    mpld3.register_plugin("responsivocustomizado", ResponsivoCustomizado);
    ResponsivoCustomizado.prototype = Object.create(mpld3.Plugin.prototype);
    ResponsivoCustomizado.prototype.constructor = ResponsivoCustomizado;
    function ResponsivoCustomizado(fig, props) {
        mpld3.Plugin.call(this, fig, props);
    }

    ResponsivoCustomizado.prototype.draw = function() {
        var fig = this.fig;
        var coords = fig.canvas.append("text").attr("class", "mpld3-coordinates").style("text-anchor", "end").style("font-size", this.props.fontsize).attr("x", this.fig.width - 5).attr("y", this.fig.height - 5);

        var refHeight = this.fig.height, refWidth = this.fig.width;
        var targetWidth = window.innerWidth, targetHeight = window.innerHeight;
        coords.text(refWidth + "," + refHeight + "/" + targetWidth + "," + targetHeight);
        if (targetWidth <= targetHeight) {
            this.fig.canvas.attr("width", targetWidth);
            this.fig.canvas.attr("height", targetWidth * refHeight / refWidth);
        } else {
            this.fig.canvas.attr("height", targetHeight);
            this.fig.canvas.attr("width", targetHeight * refWidth / refHeight);
        };
        d3.select(window).on("resize", function() {
            var targetWidth = window.innerWidth, targetHeight = window.innerHeight;
            coords.text(refWidth + "," + refHeight + "/" + targetWidth + "," + targetHeight);
            if (targetWidth <= targetHeight) {
                this.fig.canvas.attr("width", targetWidth);
                this.fig.canvas.attr("height", targetWidth * refHeight / refWidth);
            } else {
                this.fig.canvas.attr("height", targetHeight);
                this.fig.canvas.attr("width", targetHeight * refWidth / refHeight);
            };
        });

    };
    '''

    def __init__(self):
        self.dict_ = {'type': 'responsivocustomizado'}

In [118]:
paleta_cores = sns.color_palette("colorblind")
with sns.axes_style("whitegrid"), sns.plotting_context('notebook', font_scale = 1.5):
    fig, ax = plt.subplots(1, 1, figsize = (10, 10))

    col = 'Rendimento_Medio_por_Pessoa_500m'
    gdf_plot.plot(ax = ax, column = col, 
                  cmap = 'Blues',
                  lw = 0, 
                  legend = True)
    ax.set_title(col)

    #Contornos Internos
    #gdf_plot[['NM_DISTRIT', 'geometry']].dissolve('NM_DISTRIT')['geometry'].boundary.plot(ax = ax, color = 'grey', lw = 0.5)
    #gdf_plot[['NM_DISTRIT', 'geometry']].dissolve('NM_DISTRIT')['geometry'].exterior.plot(ax = ax, color = 'grey', lw = 0.5)

    #Contornos Externos
    #gdf_malha_plot['geometry'].boundary.plot(ax = ax, color = 'black', lw = 1)
    gdf_malha_plot.dissolve()['geometry'].exterior.plot(ax = ax, color = 'black', lw = 1)
        
    #Remoção de Grades e Eixos
    ax.grid(False)
    frame = plt.gca()
    frame.axes.xaxis.set_ticklabels([])
    frame.axes.yaxis.set_ticklabels([])

    #Onde Estou
    #ax.scatter(x = [-47.077066751046786], y = [-22.824513385876685], color = 'black')

    plt.box(False)
    #plt.show()

plugins.clear(fig)
#plugins.connect(fig, plugins.Reset(), plugins.Zoom(), plugins.BoxZoom())
#plugins.connect(fig, plugins.Reset(), plugins.Zoom(), plugins.MousePosition())
plugins.connect(fig, MouseDoubleClick(), ZoomCustomizado(), ResponsivoCustomizado())

html_fig = mpld3.fig_to_html(fig, figid = 'fig_mapa')
with open(f'{pasta_out}/mapa_interativo.html', 'w') as file:
    file.write(html_fig)
mpld3.display()

#plt.show()